In [1]:
#loading packages
import pyodbc
import csv
import pandas as pd
import sqlalchemy as sa
import psycopg2
import string

In [2]:
#local file location
fileLoc = r'C:\Users\hcole\Desktop\cfbstats-com-2007-1-5-0\\'

#loading files into dataframes
dfConference = pd.read_csv(fileLoc + 'conference.csv')
dfStadium = pd.read_csv(fileLoc + 'stadium.csv')
dfTeam = pd.read_csv(fileLoc + 'team.csv')
dfGame = pd.read_csv(fileLoc + 'game.csv')
dfGameStatistics = pd.read_csv(fileLoc + 'game-statistics.csv')
dfPlayer = pd.read_csv(fileLoc + 'player.csv')
dfPlayerGameStatistics = pd.read_csv(fileLoc + 'player-game-statistics.csv')
dfTeamGameStatistics = pd.read_csv(fileLoc + 'team-game-statistics.csv')
dfDrive = pd.read_csv(fileLoc + 'drive.csv')
dfPlay = pd.read_csv(fileLoc + 'play.csv')
dfPunt = pd.read_csv(fileLoc + 'punt.csv')
dfPuntReturn = pd.read_csv(fileLoc + 'punt-return.csv')
dfReception = pd.read_csv(fileLoc + 'reception.csv')
dfRush = pd.read_csv(fileLoc + 'rush.csv')
dfKickoff = pd.read_csv(fileLoc + 'kickoff.csv')
dfKickoffReturn = pd.read_csv(fileLoc + 'kickoff-return.csv')
dfPass = pd.read_csv(fileLoc + 'pass.csv')

#dataframe file array
dfFileNameCleaned = ['conference_cleaned', 'stadium_cleaned', 'stadiumdetails_cleaned', 'team_cleaned', 'teamdetails_cleaned', 'game_cleaned',
                     'gamestatistics_cleaned', 'player_cleaned', 'playerdetails_cleaned', 'playergamestatistics_cleaned',
                     'teamgamestatistics_cleaned', 'drive_cleaned', 'play_cleaned', 'punt_cleaned', 'puntreturn_cleaned', 'reception_cleaned',
                     'rush_cleaned', 'kickoff_cleaned', 'kickoffreturn_cleaned', 'pass_cleaned']

dfTableNames = ['conference', 'stadium', 'stadiumdetails', 'team', 'teamdetails', 'game', 'gamestatistics', 'player', 'playerdetails',
                'playergamestatistics', 'teamgamestatistics', 'drive', 'play', 'punt', 'puntreturn', 'reception', 'rush', 'kickoff',
                'kickoffreturn', 'pass']

#cleaning dataframes
for df in (dfConference, dfStadium, dfTeam, dfGame, dfGameStatistics, dfPlayer, dfPlayerGameStatistics, dfTeamGameStatistics, dfDrive, dfPlay, dfPunt, dfPuntReturn, dfReception, dfRush, dfKickoff, dfKickoffReturn, dfPass):
    df.columns = df.columns.str.replace('[^\w\s]', '')
    df.columns = df.columns.str.replace(' ', '')
    df.columns = df.columns.str.replace('1st', 'First')
    df.columns = df.columns.str.lower()

dfStadium['remove'] = False
dfTeam['remove'] = False
dfPlayer['remove'] = False

#creating database connection string
conn = psycopg2.connect("host=capstonealaindexing2018.postgres.database.azure.com dbname=test2 user=hstandeffer@capstonealaindexing2018 password=Alaindexing!")
cur = conn.cursor()

In [3]:
cur.execute('SELECT COUNT(*) FROM stadium')
result = cur.fetchone()
if (result[0] > 0):
    cur.execute('SELECT conferencecode FROM conference')
    conferenceCodes = cur.fetchall()
    for i, code in enumerate(conferenceCodes):
        conferenceCodes[i] = code[0]
    dfConference = dfConference[~dfConference['conferencecode'].isin(conferenceCodes)]
    
    dfStadiumResult = pd.read_sql_query('SELECT * FROM stadiumdetails', conn)
    dfStadiumResult['new'] = 0
    dfStadium['new'] = 1
    dfStadiumNew = dfStadiumResult.append(dfStadium)
    dfStadiumNew = dfStadiumNew.drop_duplicates(subset = ['name', 'city', 'state', 'capacity', 'surface'])
    duplicateStadium = dfStadiumNew.duplicated(subset = ['stadiumcode'])
    dfStadiumNew['remove'] = duplicateStadium.values
    dfStadium = dfStadiumNew[dfStadiumNew.new == 1]
    dfStadium.yearopened = dfStadium.yearopened.astype(int)
    
    dfTeamResult = pd.read_sql_query('SELECT * FROM teamdetails', conn)
    dfTeamResult['new'] = 0
    dfTeam['new'] = 1
    dfTeamNew = dfTeamResult.append(dfTeam)
    #dfTeamNew = dfTeamNew.drop_duplicates(subset = ['conferencecode'])
    duplicateTeam = dfTeamNew.duplicated(subset = ['teamcode'])
    dfTeamNew['remove'] = duplicateTeam.values
    dfTeam = dfTeamNew[dfTeamNew.new == 1]
    
    dfPlayerResult = pd.read_sql_query('SELECT * FROM playerdetails', conn)
    dfPlayerResult['new'] = 0
    dfPlayer['new'] = 1
    dfPlayerResult['playercode'] = pd.to_numeric(dfPlayerResult['playercode'])
    dfPlayerNew = dfPlayerResult.append(dfPlayer)
    #dfPlayerNew['playercode'] = pd.to_numeric(dfPlayerNew['playercode'])
    #dfPlayerNew = dfPlayerNew.drop_duplicates(subset = ['teamcode', 'uniformnumber', 'class', 'position' , 'height', 'weight', 'lastschool'])
    #dfPlayerNew = dfPlayerNew.drop_duplicates(subset = ['teamcode', 'uniformnumber', 'class', 'position'])
    duplicatePlayer = dfPlayerNew.duplicated(subset = ['playercode'])
    dfPlayerNew['remove'] = duplicatePlayer.values
    dfPlayer = dfPlayerNew[dfPlayerNew.new == 1]

C:\Users\hcole\Miniconda3\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [4]:
dfStadiumDetails = dfStadium[['name', 'city', 'state', 'capacity', 'surface', 'stadiumcode']]
dfStadium = dfStadium[['stadiumcode', 'yearopened', 'remove']]
dfStadium = dfStadium[dfStadium.remove == False]
dfStadium = dfStadium.drop(['remove'], axis=1)

dfTeamDetails = dfTeam[['teamcode', 'conferencecode']]
dfTeam = dfTeam[['teamcode', 'name', 'remove']]
dfTeam = dfTeam[dfTeam.remove == False]
dfTeam = dfTeam.drop(['remove'], axis=1)

dfPlayerDetails = dfPlayer[['playercode', 'teamcode', 'uniformnumber', 'class', 'position', 'height', 'weight', 'lastschool']]
dfPlayer = dfPlayer[['playercode', 'lastname', 'firstname', 'hometown', 'homestate', 'homecountry', 'remove']]
dfPlayer = dfPlayer[dfPlayer.remove == False]
dfPlayer = dfPlayer.drop(['remove'], axis=1)

#cleaning names by removing punctuation in names to avoid issues with csv formatting
dfPlayer['firstname'] = dfPlayer['firstname'].str.replace('[^\w\s]','')
dfPlayer['lastname'] = dfPlayer['lastname'].str.replace('[^\w\s]','')
dfPlayerDetails['lastschool'] = dfPlayerDetails['lastschool'].replace('[^\w\s]','')

dfPass['passerplayercode'] = dfPass['passerplayercode'].astype(int)
dfPass['receiverplayercode'] = dfPass['passerplayercode'].astype(int)


dfFinalArray = [dfConference, dfStadium, dfStadiumDetails, dfTeam, dfTeamDetails, dfGame, dfGameStatistics, dfPlayer, dfPlayerDetails,
                dfPlayerGameStatistics, dfTeamGameStatistics, dfDrive, dfPlay, dfPunt, dfPuntReturn, dfReception, dfRush, dfKickoff, dfKickoffReturn, dfPass]

In [5]:
for df, file, table in zip(dfFinalArray, dfFileNameCleaned, dfTableNames):
    print(df.shape)
    print(file)
    print(table)
    
    df.to_csv(r'C:\Users\hcole\Desktop\cleaned\\' + file + '.csv', index=False)
    fileString = r'C:\Users\hcole\Desktop\cleaned\\' + file + '.csv'
    with open(fileString, 'r') as f:
    # Notice that we don't need the `csv` module.
        next(f)  # Skip the header row.
        cur = conn.cursor()
        cur.copy_from(f, table, sep=',')
        conn.commit()

(1, 3)
conference_cleaned
conference
(4, 2)
stadium_cleaned
stadium
(12, 6)
stadiumdetails_cleaned
stadiumdetails
(2, 2)
team_cleaned
team
(242, 2)
teamdetails_cleaned
teamdetails
(792, 6)
game_cleaned
game
(792, 3)
gamestatistics_cleaned
gamestatistics
(7318, 6)
player_cleaned
player
(18343, 8)
playerdetails_cleaned
playerdetails


DataError: extra data after last expected column
CONTEXT:  COPY playerdetails, line 1077: "76423,66,3,JR,QB,73.0,191.0,"Bishop O'Connell HS (Arlington, VA)""
